In [24]:
import pandas as pd
import nltk
import re
import math
import timeit
from numpy import nan
import sqlalchemy

In [25]:
path = "/Users/nathan/Documents/Stats/Data Analysis/"
destination = "/Users/nathan/Documents/Stats/SignalDataScience/finalproject/datafiles/"
global debug 
debug = False
database = destination + "reviews.db"
disk_engine = sqlalchemy.create_engine('sqlite:///' + database)

In [26]:
def query_db(query, disk_engine):  
    start_time = timeit.default_timer()
    df = pd.read_sql_query(query, disk_engine)
    time_e = round((int(timeit.default_timer() - start_time))/60,2)
    if debug == True:
        print("\rQuery time: ", time_e, "minutes", end="")
    return df

In [27]:
# Make list of target product asins
query = """
SELECT x.asin, AVG(x.overall) as avg_rating FROM (
 SELECT asin, COUNT(asin) as cnt, overall 
 FROM review_full 
 GROUP BY asin
 ) x
 WHERE cnt>=500 
 GROUP BY asin
"""

target_df = query_db(query, disk_engine)
print(target_df.shape,"\n", target_df.head())
# I am arbitrarily picking my cutoff for "how many reviews a product must have"
# A better solution: look at when avg rating stabilizes (mostly) and choose that.
# Or at when sorting into 4+ star vs <4 star stabilizes

(1070, 2) 
          asin  avg_rating
0  0000031887         5.0
1  000100039X         5.0
2  0002007770         3.0
3  0002051850         5.0
4  0002185385         5.0


In [ ]:
query = "SELECT * FROM review_full WHERE asin IN '" + target_df['asin'].tolist() + "' ORDER BY unixReviewTime ASC LIMIT 20"



In [28]:
# make review_subset table with first 20 reviews for each target asin
# This method is painfully slow, I can do better
# If I can make an SQL query which gives me all results at once
asin_count = 0
first_loop = True
for asin in target_df['asin'].tolist():
    query = "SELECT * FROM review_full WHERE asin = '" + asin + "' ORDER BY unixReviewTime ASC LIMIT 20"
    temp_df = query_db(query, disk_engine)
    if temp_df.shape[0] == 20:
        temp_df.drop('helpful', axis =1, inplace=True)
        avg_rating = target_df.ix[target_df['asin']==asin, 'avg_rating']
        temp_df['avg_rating'] = float(avg_rating)
        if first_loop == True:
            first_loop = False
            temp_df.to_sql('review_subset',
                            disk_engine,
                            if_exists='replace',
                            index= False,
                            dtype={'asin':sqlalchemy.types.NVARCHAR(length=10),
                             'overall':sqlalchemy.types.INTEGER(),
                             'reviewTime':sqlalchemy.NVARCHAR(length=10),
                             'reviewerID':sqlalchemy.types.NVARCHAR(length=14),
                             'unixReviewTime':sqlalchemy.types.INTEGER(),
                             'reviewerName':sqlalchemy.types.NVARCHAR(length=30),
                             'vader':sqlalchemy.types.Float(precision=4, asdecimal=True),
                             'avg_rating':sqlalchemy.types.Float(precision=3, asdecimal=True)})
        else:
            temp_df.to_sql('review_subset',
                            disk_engine,
                            if_exists='append',
                            index= False,
                            dtype={'asin':sqlalchemy.types.NVARCHAR(length=10),
                             'overall':sqlalchemy.types.INTEGER(),
                             'reviewTime':sqlalchemy.NVARCHAR(length=10),
                             'reviewerID':sqlalchemy.types.NVARCHAR(length=14),
                             'unixReviewTime':sqlalchemy.types.INTEGER(),
                             'reviewerName':sqlalchemy.types.NVARCHAR(length=30),
                             'vader':sqlalchemy.types.Float(precision=4, asdecimal=True),
                             'avg_rating':sqlalchemy.types.Float(precision=3, asdecimal=True)})
        asin_count += 1
        print("\r", asin_count, "products included", end="")


1 products included





















































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































In [29]:
# Export review_subset left join on reviewer_scores to csv

query = """
SELECT DISTINCT * FROM (review_subset a
INNER JOIN (SELECT reviewerID, SUM(reviewerScore)
FROM reviewer_scores
GROUP BY reviewerID
) u
ON a.reviewerID = u.reviewerID)
"""
final_df = query_db(query, disk_engine)
print(final_df.head())
final_df.to_csv(destination + "reviewsummary.csv")

         asin  overall   reviewTime             reviewerID  unixReviewTime  \
0  0062059963        3  04 23, 2013  A06569781Q9EFQMBU0NWY      1366675200   
1  0199571120        2  10 26, 2012  A06797061Q7VQX2TZGIX8      1351209600   
2  0062014536        4   03 5, 2013   A0854565P15JI2ZXLBPO      1362441600   
3  014242899X        4  01 11, 2011         A1003CRD5N6PZJ      1294704000   
4  0099911701        5   02 1, 1998         A100M1123T9P5Z       886291200   

           reviewerName   vader  avg_rating             reviewerID  \
0                Ashley -0.9039         5.0  A06569781Q9EFQMBU0NWY   
1                Morgan  0.6597         5.0  A06797061Q7VQX2TZGIX8   
2            Carly Rude  0.6917         2.0   A0854565P15JI2ZXLBPO   
3             Aimee Pan  1.1796         2.0         A1003CRD5N6PZJ   
4  chyler@ix.netcom.com  3.0471         1.0         A100M1123T9P5Z   

   SUM(reviewerScore)  
0                 -80  
1                 -20  
2                 -20  
3             